# Game

## Setup

* The game goes for N weeks, starting at the year start. After N weeks rewards are computed. 
* The agent knows all information about historically realisations of sales.
* There is seasonal stock, which has a certain residual value at the end of N weeks (known) 
* New stock appears from time to time -> if article season is now => article appears


## CSKU features

* Black Price
* Initial Stock
* Current Stock
* COGS
* Season (1-4)


## Action 
Agent should define weekly discounts for the currently available stock, knowing previous history and number of weeks left till end of financial year, if sales are on


## Reward

Revenue, if profit> x% of Revenue
Otherwise: Revenue - penalty * (profit undershoot) 





Demand depends on

1. price
2. competitor price
3. seasonal effects
4. sale event = marketing 
5. market shock



Demand 

In [1]:
import numpy as np
import pandas as pd
from typing import Tuple
from data.generator import SimpleDemandGenerator
from game.simulator import PricingGame
import plotly.graph_objects as go


In [2]:

n = 100
generator = SimpleDemandGenerator()
simulator = PricingGame(generator, n)



In [3]:
for i in range(52):
    simulator.play_prices(discounts= np.random.choice([0,20,25,30,35,40],n))


In [4]:
def plot_pricing_game(simulator):
    sales, stock, revenue, profit,sdrs = simulator.get_kpis()
    
    summed_revenue = [r.sum() for r in revenue]
    summed_profit = [p.sum() for p in profit]
    x = list(range(52))
    trace1 = go.Scatter(x=x, y=summed_revenue, mode='lines', name='Revenue')
    trace2 = go.Scatter(x=x, y=summed_profit, mode='lines', name='Profits')

    fig = go.Figure(data=[trace1, trace2])
    fig.update_layout(title='Revenue and Profit', xaxis_title='Calendar Week', yaxis_title='Euros')
    fig.update_traces(line=dict(width=2))
    fig.show()
    
    
    fig = go.Figure(data=go.Scatter(x=x, y=sdrs, mode='lines'))
    fig.update_layout(title='SDR', xaxis_title='Calendar Week', yaxis_title='SDR')
    fig.update_traces(line=dict(width=2))
    fig.show()

In [5]:
import plotly.graph_objects as go

x = [1, 2, 3, 4, 5]


fig = go.Figure(data=go.Scatter(x=list(range(52)), y=[simulator.stocks[i][0] for i in range(52)], mode='lines'))
fig.update_layout(title='My Line Plot', xaxis_title='Calendar Week', yaxis_title='Stock')
fig.update_traces(line=dict(width=2))
fig.show()

In [6]:
plot_pricing_game(simulator)

In [7]:
simulator.get_final_score()

(2810418.7460757373,
 3792950.970850358,
 91394.32606505588,
 982532.2247746203,
 3605868.221882115,
 13489.161003567577,
 187082.7489682425,
 77905.1650614883)

In [8]:
self = simulator
revenue = sum([r.sum() for r in self.revenues])
profit = sum([r.sum() for r in self.profits])

residual_revenue = (self.stocks[-1] * self.black_prices * self.residual_value).sum()
residual_profit = (self.stocks[-1] * (self.black_prices * self.residual_value - self.cogs)).sum()

total_revenue = revenue + residual_revenue
total_profit = profit + residual_profit

penalty = max(0, total_revenue * self.target_profit_ratio - total_profit) * self.profit_lack_penalty

score = total_revenue - penalty